In [1]:
#Initialize dependencies
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorboard as tb
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)

import config.names as names
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, trainer_config, metrics_config
from modules.tf_text_postprocess import TextPostprocessor
from modules.datagen import DataGenerator
from modules.train_step import Epoch
def save_weights_error(model):
    model = model
    %store model
    return 

# Config training

In [2]:
# Configure the Transformer layer
config = transformer_config()
config_train = trainer_config()
config_metrics = metrics_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])

model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer)

regularized_layer = RegularizedDenseLayer(config["d_model"])

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=config_train['loss_reduction'])
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=config_train['learning_rate'])

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
           tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
           tf.keras.metrics.Precision(thresholds=0.5, name=config_metrics['precision']),]

model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)



In [3]:
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, names.TRAIN_DATASET_END_DIR)
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, names.VALID_DATASET_END_DIR)

datagen = DataGenerator(TRAIN_DATASET_DIR, VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()

{'russian': [<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[1],
       [2]], dtype=int32)>], 'english': [<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [4]], dtype=int32)>]}
{'russian': [<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[1]], dtype=int32)>], 'english': [<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[2]], dtype=int32)>]}
Train data info: 2 classes, 2 samples
Valid data info: 2 classes, 2 samples


In [4]:
epochs = config_train['epoch_train']
model.fit_model(train_english, 
                         train_russian,
                         valid_english,
                         valid_russian,
                         epochs=epochs,
                         save_model_each_epoch=True,
                         logs=True,
                         model_name='novelsdreamer-r')


2023-10-19 15:40:03.991684: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to int32 is not supported


UnimplementedError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to int32 is not supported [Op:Cast]